Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:
The goal of this lab is:
- Learn how to use CUDA C and CUDA Fortran to parallelize our code.
- Understand the basic terms and steps involved in making a sequential code parallel.

We do not intend to cover:
- Optimization techniques like memory access patterns, memory hierarchy.

# Introduction
Graphics Processing Units (GPUs) were initially designed to accelerate graphics processing, but in 2007 the release of CUDA introduced GPUs as General Purpose Processors. CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple and elegant. The developer still programs in the familiar C, C++, Fortran, or an ever expanding list of supported languages, and incorporates extensions of these languages in the form of a few basic keywords.

- CUDA C/C++ is based on a standard C/C++ and CUDA Fortran is based on a standard Fortran
- CUDA is a set of extensions to enable heterogeneous programming
- CUDA is a straightforward API to manage devices, memory, etc.


# CUDA 


**Heterogeneous Computing:** CUDA is a heterogeneous programming model that includes provisions for both a CPU and GPU. The CUDA C/C++ programming interface consists of C language extensions and the CUDA Fortran programming interface consists of Fortran language extensions. These enables you to target portions of source code for parallel execution on the device (GPU). CUDA provides library of C/Fortran functions that can be
executed on the host (CPU) so that it can interact with the device. The two processor that work with each other are: 

- Host: CPU and its memory (Host Memory)
- Device: GPU and its memory  (Device Memory)


Let us look at a Hello World example in C and Fortran: 


<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>
    
```cpp
_global__ void print_from_gpu(void) {
    printf("Hello World! from thread [%d,%d] From device\n", threadIdx.x,blockIdx.x);
}

int main(void) {
    printf("Hello World from host!\n");
    print_from_gpu<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

```
</details>
<br/>


<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>
    

```fortran
module printgpu
contains
  attributes(global) subroutine print_form_gpu()
    implicit none
    integer :: i
    i = blockDim%x * (blockIdx%x - 1) + threadIdx%x
    print *, i
  end subroutine saxpy 
end module printgpu

program testPrint
  use printgpu
  use cudafor
  implicit none

  call print_form_gpu<<<1, 1>>>()
  cudaDeviceSynchronize()
end program testPrint

```
</details>
<br/>


So you might have already observed that CUDA C is nothing but extensions/constructs to existing language. Let us look at what those additional constructs we introduced above:

- ```__global__``` :This keyword, when added before the function, tells the compiler that this is a function that will run on the device and not on the host. 
- ``` <<<,>>> ``` : This keyword tells the compiler that this is a call to the device function and not the host function. Additionally, the 1,1 parameter basically dictates the number of threads to launch in the kernel. We will cover the parameters inside angle brackets later. 
- ``` threadIdx.x, blockIdx.x ``` : This is a unique ID that's given to all threads. 
- ``` cudaDeviceSynchronize() ``` : All of the kernel(Function that runs on GPU) calls in CUDA are asynchronous in nature. This API will make sure that host does not proceed until all device calls are over.


## GPU Architecture
 
In this section will take an approach of describing the CUDA programming model by showing relationship between the software programming concepts and how do they get mapped to GPU hardware.

The diagram below shows a higher level of abstraction of components of GPU hardware and its respective programming model mapping. 

<img src="../../_common/images/cuda_hw_sw.png" width="80%" height="80%">

As shown in the diagram above CUDA programming model is tightly coupled with hardware design. This makes CUDA one of the most efficient parallel programming model for shared memory systems. Another way to look at the diagram shown above is given below: 

| Software | Executes  | Hardware |
| --- | --- | --- |
| CUDA thread  | on/as | CUDA Core | 
| CUDA block  | on/as | Streaming Multiprocessor |
| GRID/Kernel  | on/as | GPU Device |

We will get into the concept of _blocks_ and _threads_ in upcoming section. But let us first look at steps involved in writing CUDA code.


## Steps in CUDA Programming

The below table highlights the typical steps which are required to convert sequential code to CUDA code:

| Sequential code | CUDA Code |
| --- | --- |
| **Step 1** Allocate memory on the CPU ( _malloc new_ ) | **Step 1** : Allocate memory on the CPU (_malloc, new_ )|
| **Step 2** Populate/initialize the CPU data | **Step 2** Allocate memory on the GPU, using API like _cudaMalloc()_ |
| **Step 3** Call the CPU function that has the crunching of data. | **Step 3**  Populate/initialize the CPU  |
| **Step 4** Consume the crunched data on Host | **Step 4** Transfer the data from the host to the device with _cudaMemcpy()_ |
| | **Step 5** Call the GPU function with _<<<,>>>_ brackets |
| | **Step 6** Synchronize the device and host with _cudaDeviceSynchronize()_ |
| | **Step 7** Transfer data from the device to the host with _cudaMemcpy()_ |
| | **Step 8** Consume the crunched data on Host |

CPU and GPU memory are different and developer needs to use additional CUDA API to allocate and free memory on GPU. Only device memory can be consumed inside GPU function call (kernel). 
    
In CUDA C/C++, linear memory on Device is typically allocated using ```cudaMalloc()``` and freed using ```cudaFree()``` and data transfer between host memory and device memory are typically done using ```cudaMemcpy()```.

In CUDA Fortran, linear memory on Device is typically allocated by defining array as  ```allocatable, device``` type and data transfer between host memory and device memory are typically done using ```cudaMemcpy()```.
    

The API definition of these are as follows: 

**cudaError_t cudaMalloc (void ∗∗ devPtr, size_t size)** in CUDA C/C++ and **integer function cudaMalloc(devptr, size)**  in CUDA Fortran, allocate size bytes of linear memory on the device and returns a pointer to the allocated memory. The allocated memory is suitably aligned for any kind of variable. `cudaMalloc()` returns ```cudaErrorMemoryAllocation``` in case of failure or ```cudaSuccess```.
    
**cudaError_t cudaMemcpy (void ∗ dst, const void ∗ src, size_t count, enum cudaMemcpyKind kind)** in CUDA C/C++ and  **integer function cudaMemcpy(dst, src, count, kind)** in CUDA Fortran, copies count bytes from the memory area pointed to by `src` to the memory area pointed to by `dst`. `dst` and `src` may be any device or host, scalar or array.  `kind` is one of the defined enums `cudaMemcpyHostToDevice`, `cudaMemcpyDeviceToHost`, `cudaMemcpyDeviceToDevice` or `cudaMemcpyHostToHost` (this specifies the direction of the copy).

Please note, calling `cudaMemcpy()` with `dst` and `src` pointers that do not match the direction of the copy results in an undefined behavior.

**cudaError_t cudaFree (void ∗ devPtr)** Frees the memory space pointed to by `devPtr`, which must have been returned by a previous call to `cudaMalloc()` or other equivalent API. 
    
Let us look at these steps in more detail for a simple vector addition code:

    
<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>
    
```cpp
int main(void) {
	int *a, *b, *c;
        int *d_a, *d_b, *d_c; // device copies of a, b, c

	int size = N * sizeof(int);

	// Alloc space for host copies of a, b, c and setup input values
	a = (int *)malloc(size); fill_array(a);
	b = (int *)malloc(size); fill_array(b);
	c = (int *)malloc(size);

        // Alloc space for device copies of a, b, c
        cudaMalloc((void **)&d_a, size);
        cudaMalloc((void **)&d_b, size);
        cudaMalloc((void **)&d_c, size);

       // Copy inputs to device
        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);


	device_add<<<N,1>>>(d_a,d_b,d_c);

        // Copy result back to host
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

	print_output(a,b,c);

	free(a); free(b); free(c);
        cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);



	return 0;
}
```
</details>
<br/>


<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>
    

```fortran
module kernel
    contains
    ! CUDA kernel. Each thread takes care of one element of c
    attributes(global) subroutine vecAdd_kernel(n, a, b, c)
        integer, value :: n
        real(8), device :: a(n), b(n), c(n)
        integer :: id
 
        ! Get our global thread ID
        id = (blockidx%x-1)*blockdim%x + threadidx%x
 
        ! Make sure we do not go out of bounds
        if (id <= n) then
            c(id) = a(id) + b(id)
        endif
    end subroutine vecAdd_kernel
end module kernel
 
program main
    use cudafor
    use kernel
 
    type(dim3) :: blockSize, gridSize
    real(8) :: sum
    integer :: i
 
    ! Size of vectors
    integer :: n = 1
 
    ! Host input vectors
    real(8),dimension(:),allocatable :: h_a
    real(8),dimension(:),allocatable :: h_b
    !Host output vector
    real(8),dimension(:),allocatable :: h_c
 
    ! Device input vectors
    real(8),device,dimension(:),allocatable :: d_a
    real(8),device,dimension(:),allocatable :: d_b
    !Host output vector
    real(8),device,dimension(:),allocatable :: d_c
 
    ! Allocate memory for each vector on host
    allocate(h_a(n))
    allocate(h_b(n))
    allocate(h_c(n))
 
    ! Allocate memory for each vector on GPU
    allocate(d_a(n))
    allocate(d_b(n))
    allocate(d_c(n))
 
    ! Initialize content of input vectors, vector a[i] = sin(i)^2 vector b[i] = cos(i)^2
    do i=1,n
        h_a(i) = sin(i*1D0)*sin(i*1D0)
        h_b(i) = cos(i*1D0)*cos(i*1D0)
    enddo
 
    ! Implicit copy of host vectors to device
    d_a = h_a(1:n)
    d_b = h_b(1:n)
 

    ! Execute the kernel
    call vecAdd_kernel<<<1, 1>>>(n, d_a, d_b, d_c)
 
    ! Implicit copy of device array to host
    h_c = d_c(1:n)
 
    ! Sum up vector c and print result divided by n, this should equal 1 within error
    sum = 0.0;
    do i=1,n
        sum = sum +  h_c(i)
    enddo
    sum = sum/real(n)
    print *, 'final result: ', sum
 
    ! Release device memory
    deallocate(d_a)
    deallocate(d_b)
    deallocate(d_c)
 
    ! Release host memory
    deallocate(h_a)
    deallocate(h_b)
    deallocate(h_c)
 
end program main
```

</details>
<br/>


### Unified Memory
An easier way to allocate memory accessible by the GPU is to use *Unified Memory*. It provides a single memory space accessible by all GPUs and CPUs in the system. To allocate data in unified memory, we call `cudaMallocManaged()`, which returns a pointer that you can access from host (CPU) code or device (GPU) code. To free the data, just pass the pointer to `cudaFree()`. To read more about unified memory, please checkout the blog on [Unified Memory for CUDA beginners](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/).

<img src="../../_common/images/unified_memory.png">

Below is the example usage of how to use managed memory in the CUDA code:

<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>

```cpp
 // Allocate Unified Memory -- accessible from CPU or GPU
  int *a, *b, *c;
  cudaMallocManaged(&a, N*sizeof(int));
  cudaMallocManaged(&b, N*sizeof(int));
  cudaMallocManaged(&c, N*sizeof(int));
  ...

  // Free memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
```
</details>
<br/>


<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>
    
```fortran
!matrix data
real, managed, allocatable, dimension(:,:) :: A, B, C
```
</details>
<br/>

## Understanding Threads and Blocks
We will be looking at understanding _thread_ and _block_ level parallelism in this section.The number of threads and blocks to be launched is passed as parameter to ```<<<,>>>``` brackets in a kernel call.

### Creating multiple blocks

In order to create multiple blocks for vector addition code above you need to change two things:
1. Change _<<<1,1>>>_ to <<<N,1>>>_ which basically launches N number of blocks
2. Access the array with block index using private variable passed by default to CUDA kernel: _blockIdx.x_


<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>
    
```cpp
//changing from device_add<<<1,1>>> to
device_add<<<N,1>>>
//access the array using blockIdx.x private variable
__global__ void device_add(int *a, int *b, int *c) {
    c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}
```
</details>
<br/>


<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>

```fortran
attributes(global) subroutine vecAdd_kernel(n, a, b, c)
        integer, value :: n
        real(8), device :: a(n), b(n), c(n)
        integer :: id
 
        ! Get our global thread ID
        id = blockidx%x
 
        ! Make sure we do not go out of bounds
        if (id <= n) then
            c(id) = a(id) + b(id)
        endif
    end subroutine vecAdd_kernel
}
```  

</details>
<br/>

By using `blockIdx.x` to index the array, each block handles a different element of the array and may execute in parallel to each other.

| Block Id | Performs |
| --- | --- |
| Block 0 | _c\[0\]=b\[0\]+a\[0\]_ |
| Block 1 | _c\[1\]=b\[1\]+a\[1\]_ |
| Block 2 | _c\[2\]=b\[2\]+a\[2\]_ |

**Understand and analyze** the sample vector addition code [vector_addition_block.cu](../source_code/vector_addition_gpu_block_only.cu).Open the downloaded files for inspection. 



### Creating multiple threads

In order to create multiple threads for vector addition code above. You need to change two things:
1. change _<<<1,1>>>_ to <<<1,N>>>_ which basically launches N number of threads inside 1 block
2. Access the array with thread index using private variable passed by default to CUDA kernel: _threadIdx.x_


<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>
    
```cpp
//changing from device_add<<<1,1>>> to
device_add<<<1,N>>>
//access the array using threadIdx.x private variable
__global__ void device_add(int *a, int *b, int *c) {
    c[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x];
}
```
</details>
<br/>


<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>
    
 ```fortran
attributes(global) subroutine vecAdd_kernel(n, a, b, c)
        integer, value :: n
        real(8), device :: a(n), b(n), c(n)
        integer :: id
 
        ! Get our global thread ID
        id = threadidx%x
 
        ! Make sure we do not go out of bounds
        if (id <= n) then
            c(id) = a(id) + b(id)
        endif
    end subroutine vecAdd_kernel
```   

</details>
<br/>
    
By using `threadIdx.x` to index the array, each thread handles a different element of the array and can execute in parallel.

| thread Id | Performs |
| --- | --- |
| Thread 0 | _c\[0\]=b\[0\]+a\[0\]_ |
| Thread 1 | _c\[1\]=b\[1\]+a\[1\]_ |
| Thread 2 | _c\[2\]=b\[2\]+a\[2\]_ |

**Understand and analyze** the sample vector addition code [vector_addition_thread.cu](../source_code/vector_addition_gpu_thread_only.cu).
    
### Creating multiple blocks each having many threads

So far, we've looked at parallel vector addition through the use of several blocks with one thread and one block with several
threads. Now let us look at creating multiple blocks, each block containing multiple threads.

To understand it lets take a scenario where the total number of vector elements is 32 which needs to be added in parallel. Total number of parallel execution unit required is 32. As a first step let us define that each block contains eight threads(we are not saying this is optimal configuration and is just for explanation purpose). Next we define the number of blocks. The simplest calculation is No_Of_Blocks = 32/8 where 8 is number of threads per blocks. The code changes required to launch 4 blocks with 8 thread each is as shown below: 
1. Change _<<<1,1>>>_ to <<<4,8>>>_ which basically launches 4 number of threads per block and 8 total blocks
2. Access the array with both thread index and block index using private variable passed by default to call CUDA kernel: _threadIdx.x_ and _blockIdx.x_ and _bloxkDim.x_ which tells how many threads are allocated per block. 

    
<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>

```cpp
threads_per_block = 8;
no_of_blocks = N/threads_per_block;
device_add<<<no_of_blocks,threads_per_block>>>(d_a,d_b,d_c);

__global__ void device_add(int *a, int *b, int *c) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    c[index] = a[index] + b[index];
}
```
</details>
<br/>

<details>
<summary markdown="span"><b>CUDA Fortran</b></summary>
    
```fortran
! Number of threads in each thread block
     blockSize = dim3(8,1,1)
     ! Number of thread blocks in grid
     gridSize = dim3(ceiling(real(n)/real(blockSize%x)) ,1,1)
     call vecAdd_kernel<<<gridSize, blockSize>>>(n, d_a, d_b, d_c)

    ! CUDA kernel. Each thread takes care of one element of c
    attributes(global) subroutine vecAdd_kernel(n, a, b, c)
        integer, value :: n
        real(8), device :: a(n), b(n), c(n)
        integer :: id
 
        ! Get our global thread ID
        id = (blockidx%x-1)*blockdim%x + threadidx%x
 
        ! Make sure we do not go out of bounds
        if (id <= n) then
            c(id) = a(id) + b(id)
        endif
    end subroutine vecAdd_kernel
```
</details>
<br/>
    
The diagram below shows the launch configuration that we discussed so far:

<img src="../../_common/images/cuda_indexing.png">

Modern GPU Architectures consists of multiple SM, each consisting of number of cores. In order to utilize whole GPU it is important to make use of both threads and blocks. 

**Understand and analyze** the sample vector addition code [vector_addition_block_thread.cu](../source_code/vector_addition_gpu_thread_block.cu).Open the downloaded files for inspection. 


The more important question which may arise is why bother with threads altogether? What do we gain by adding additional level of parallelism? Short answer is CUDA programming model defines that unlike parallel blocks, threads have mechanisms to efficiently communicate and synchronize.

This is necessary to implement certain algorithms where threads needs to communicate with each other.We do not require synchronization across threads in **Pair Calculation** so we will not be going into details of concept of synchronization across threads and usage of specialized memory like _shared_ memory in this tutorial.  

# Atomic Construct

In the code you will also require one more construct which will help you in getting the right results. OpenACC atomic construct ensures that a particular variable is accessed and/or updated atomically to prevent indeterminate results and race conditions. In other words, it prevents one thread from stepping on the toes of other threads due to accessing a variable simultaneously, resulting in different results run-to-run. For example, if I want to count the number of elements that have a value greater than zero, we could write the following:


<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>
    
```cpp
__global__ void countMoreThanZero( ... )
{
    if ( val > 0 )
    {
        atomicAdd(&cnt[0],1);
    }
}
```
</details>
<br/>

<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>

```fortran
if(r<cut)then
         oldvalue = atomicadd(g(ind),1.0d0)
endif
```
</details>
<br/>
    
# A Quick Recap
We saw the definition of CUDA and briefly covered CUDA architecture and introduced CUDA C and CUDA Fortran constructs. We also played with block and thread configurations for a simple vector addition code. All this was done under the following restrictions:
1. **Multiple Dimension**: We launched threads and blocks in one dimension. We have been using `threadIdx.x` and `blockIdx.x`, so what is `.x` ? This statement basically says that we are launching threads and blocks in one dimension only. CUDA allows to launch threads in 3 dimensions. You can also have `.y` and `.z` for index calculation. For example you can launch threads and blocks in 2 dimensions for dividing work for a 2D image. Also the maximum number of threads per block and number of blocks allowed per dimension is restricted based on the GPU that the code is run on.
2. **GPU Memory**: What we have not covered is that GPU has different hierarchy of memory, e.g. GPU has a read only memory which provides high bandwidth for 2D and 3D locality access called _texture_. Also GPU provides a scratch pad limited memory called as _shared memory_
3. **Optimization** : What we did not cover so far is the right way to access the compute and memory to get max performance. 

**One key characteristic about CUDA is that a user can control access pattern of data for each thread. The user can decide which part of memory the data can sits on. While we are covering some part of this in this lab, which is required for us to port our code, we do not intend to cover all optimizations**

## Compile and Run for NVIDIA GPU
Now, lets start modifying the original code and add add the CUDA constructs. You can either explicitly transfer the allocated data between CPU and GPU or use unified memory which creates a pool of managed memory that is shared between the CPU and GPU.

Click on the <b>[C/C++ version](../source_code/rdf.cu)</b> or the <b>[Fortran version](../source_code/rdf.f90)</b> links, and start modifying the C or Fortran version of the RDF code. Remember to **SAVE** your code after changes, before running below cells.

**Note:** When `-arch=native` compiled option is used, `nvcc` detects the visible GPUs on the system and generates codes for them. It is a warning if no visible supported GPU on the system, and the default architecture will be used.

Moreover, for the CUDA Fortran version, we are targeting the NVTX v3 API, a header-only C library, and added Fortran-callable wrappers to the code, we add `-lnvhpcwrapnvtx` at the compile time to do the link to the library.

In [ ]:
#compile for Tesla GPU (C/C++)
!cd ../source_code && echo "compiling C/C++ version .. " && nvcc -arch=native -o rdf_c rdf.cu && echo "Running the executable and validating the output" && ./rdf_c && cat Pair_entropy.dat

<details>
    <summary markdown="span"><b>Output</b></summary>
    
    
The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```
</details>
<br/>



In [ ]:
#compile for Tesla GPU (Fortran)
!cd ../source_code && echo "compiling Fortran version .. " && nvfortran -cuda -o rdf_f rdf.f90 -lnvhpcwrapnvtx && echo "Running the executable and validating the output" && ./rdf_f && cat Pair_entropy.dat


<details>
    <summary markdown="span"><b>Output</b></summary>
    
    
The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527  
```
</details>
<br/>


Now, let's profile the code.

In [ ]:
#profile and see output of nvptx (C/C++)
!cd ../source_code && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_cuda_c ./rdf_c

In [ ]:
#profile and see output of nvptx (Fortran)
!cd ../source_code && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_cuda_f ./rdf_f

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> the [C/C++ version](../source_code/rdf_cuda_c.nsys-rep) or the  [Fortran version](../source_code/rdf_cuda_f.nsys-rep) and choosing <mark>save Link As</mark> Once done, open it via the GUI. Have a look at the example expected profiler report below:


<details>
    <summary markdown="span"><b>Example screenshot (C/C++ code)</b></summary>
    
<img src="../../_common/images/cuda_profile_timeline.png">
</details>
<br/>


<details>
    <summary markdown="span"><b>Example screenshot (Fortran code)</b></summary>
    
<img src="../../_common/images/cuda_profile_timeline.jpg">
</details>
<br/>


Nsight systems is capable of capturing information about CUDA execution in the profiled process.CUDA API row in the timeline view shows traces of CUDA Runtime and Driver calls made by application. If you hover your mouse over it, you will see more information about the calls.

   
<img src="../../_common/images/cuda_profile_api.png">


Near the bottom of the timeline row tree, the GPU node will appear and contain a CUDA node. Within the CUDA node, each CUDA context used within the process will be shown along with its corresponding CUDA streams. Streams will contain memory operations and kernel launches on the GPU. In the example screenshot below, you can see Kernel launches are represented by blue, while memory transfers are displayed in red and green. In this example screenshot, unified memory was used rather than explicitly transferring data between CPU and GPU.

<img src="../../_common/images/cuda_profile.png">


Feel free to checkout the solutions for [C/C++ solution (with managed memory)](../source_code/SOLUTION/rdf_unified_memory.cu), [C/C++ solution (without managed memory)](../source_code/SOLUTION/rdf_malloc.cu) and [Fortran solution (with managed memory)](../source_code/SOLUTION/rdf_unified_memory.f90) versions to help you understand better.

# Analysis

**Usage Scenarios**

Using launguage extensions like CUDA C, CUDA Fortran helps developers get the best performance out of their code on an NVIDIA GPU. CUDA C and other language construct exposes the GPU architecture and programming model which gives more control to developers with respect to memory storage, access and thread control. Based on the type of application it can provide many fold improvement over say compiler generated codes with help of directives. 

**How is CUDA different from other GPU progamming models like OpenACC and OpenMP?**

CUDA should not be considered an alternative to OpenMP or OpenACC. In fact CUDA complements directive-based programming models and there are defined interoperability strategies between them. You can always start accelerating your code with OpenACC and use CUDA to optimize the most performance critical kernels. For example use OpenACC for data transfer and then pass a device pointer to one of critical CUDA kernels which is written in CUDA. 

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _files.zip
zip -r _files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../_files.zip) and choosing <mark>save Link As</mark>.

-----


# Links and Resources
[Introduction to CUDA](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).